In [7]:
import networkx as nx
import numpy as np
import pandas as pd
import igraph as ig
import matplotlib.pyplot as plt

In [2]:
sfn = nx.scale_free_graph(n=1000)

In [11]:
bba = nx.barabasi_albert_graph(n=1000, m=100)

In [12]:
acc = nx.average_clustering(bba)
print(acc)

0.27173273364340966


In [13]:
sp = nx.average_shortest_path_length(bba)
print(sp)

1.8198198198198199
